In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt

N=666
num_bins=50

# I. Load and plot

In [ ]:
# Loading data
with open('first_kind.npy', 'rb') as f:
    zeroes_first_kind = np.load(f)
with open('second_kind.npy', 'rb') as f:
    zeroes_second_kind = np.load(f)
with open('critical_points.npy', 'rb') as f:
    critical_points = np.load(f)
with open('branch_points.npy', 'rb') as f:
    branch_points = np.load(f)

# Histogram of singular values
fig, ax = plt.subplots()
n, bins, patches = ax.hist(zeroes_first_kind, num_bins, density=True)
ax.set_xlabel('Eigenvalues')
ax.set_ylabel('Probability density')
ax.set_title(r'Histogram of singular values for N={}'.format(N))
fig.tight_layout()
plt.show()

In [ ]:
# Plotting critical points
plt.figure( figsize=(20,10) )
plt.scatter( np.real(critical_points), np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
plt.scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (N={N})')
plt.scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (N-1={N-1})')
plt.title( 'Eigenvalues and critical points')
plt.legend()
plt.show()

# Plotting branch points
## Transport colors to uniform
branch_points_color = np.abs( np.imag( critical_points ) )
branch_points_color /= np.max( branch_points_color )
uniform_sample = np.linspace( 0, 1, len(branch_points_color))
sorted_indices = np.argsort( branch_points_color )
branch_points_color = uniform_sample[ sorted_indices ]
## Plots as usual
plt.figure( figsize=(10,20) )
plt.scatter( np.real(branch_points),  np.imag(branch_points), marker='*', c=branch_points_color, label=f'Branch points (N-1={N-1})')
plt.scatter( np.real(branch_points), -np.imag(branch_points), marker='x', c=branch_points_color, label=f'(Conjugate) Branch points (N-1={N-1})')
plt.title( 'Branch points $m = M_\mu(z)$')
plt.legend()
plt.colorbar()
plt.show()

# II. Performance test when evaluating Stieljes transforms

Here we compare numpy vs sympy. Although it may be true that symbolic computations
using sympy might be slower, the question is to evaluate the overhead.

In [ ]:
def G_empirical(z):
    array = z[...,None]-zeroes_first_kind[...,:]
    return np.sum( 1/array, axis=-1)/len(zeroes_first_kind)

def G_prime_empirical(z):
    array = z[...,None]-zeroes_first_kind[...,:]
    return np.sum( -1/(array*array), axis=-1)/len(zeroes_first_kind)

def G_second_empirical(z):
    array = z[...,None]-zeroes_first_kind[...,:]
    return np.sum( 2/(array*array*array), axis=-1)/len(zeroes_first_kind)

def M_empirical(z):
    return z*G_empirical(z)-1

def M_prime_empirical(z):
    return z*G_prime_empirical(z) + G_empirical(z)

def M_second_empirical(z):
    return z*G_second_empirical(z) + 2*G_prime_empirical(z)

In [ ]:
import sympy as sp

z = sp.symbols("z")
sp_G_empirical = 0
for zero in zeroes_first_kind:
    sp_G_empirical = sp_G_empirical + 1/(z-zero)
sp_G_empirical = sp_G_empirical/len(zeroes_first_kind)

sp_M_empirical = z*sp_G_empirical - 1

In [ ]:
# Test
z0 = np.array( complex(1.0+1.0j) )
print(z0)
print( G_empirical(z0) )
print( sp_G_empirical.subs(z, z0).evalf() )

In [ ]:
# Benchmark
points_counts = [10, 25, 50, 100]

benchmark_results = {
    'numpy_vectorized': [],
    'numpy_loop': [],
    'sympy': []
}
for count in points_counts:
    print(f'''Running benchmark for {count} points...''')
    points = np.linspace(0, 10, count)+1.0j

    print( "  |- Numpy benchmark (in a single vectorized call)...")
    start = time.time()
    values = M_empirical( points )
    end = time.time()
    timing = end-start
    benchmark_results["numpy_vectorized"].append(timing)
    print("  |- Total time: ", timing)

    print( "  |- Numpy benchmark (in a loop)...")
    start = time.time()
    values_numpy = np.zeros_like( values )
    for i in range( len(values_numpy) ):
        values_numpy = M_empirical( points[i] )
    end = time.time()
    timing = end-start
    benchmark_results["numpy_loop"].append(timing)
    print("  |- Total time: ", timing)

    print( "  |- Sympy benchmark...")
    start = time.time()
    values_sympy = np.zeros_like( values )
    for i in range( len(values_sympy) ):
        values_sympy = sp_M_empirical.subs('z', points[i] ).evalf()
    end = time.time()
    timing = end-start
    benchmark_results["sympy"].append(timing)
    print("  |- Total time: ", timing)

    print("")
# End for

# GOSH! Huge overhead!

## III. Inversion without homotopy

In [ ]:
def newton_raphson( f, f_prime, initial, max_iter=12, tol=1e-12):
    z = initial
    k = 0
    while k<max_iter:
        value = f(z)
        derivative_value = f_prime(z)
        z = z - value/derivative_value
        if abs(value)<tol:
            break
        k = k + 1
    return z

In [ ]:
# Find the z_target such that M(z_target) = m_target
empirical_mean = zeroes_first_kind.mean()
m_array = np.linspace( 0.1, 20.0, 1000 )*(-1j) # Breaks at 0.7
z_array = []
debug = False
for m in m_array:
    m_target  = np.array( [m] )
    if len(z_array)>0:
        z_initial = z_array[-1]
    else:
        z_initial = empirical_mean/m_target
    z_result  = newton_raphson( lambda z : M_empirical(z)-m_target, M_prime_empirical, z_initial)
    z_array.append( z_result )
    if debug:
        print( "Result: ", z_result)
        print( "Error: ", np.abs( m_target - M_empirical(z_result)) )
z_array = np.array( z_array )

m_final = m_array[-1]
z_final = z_array[-1]

# Plots in z and m space
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

axs[0].plot( np.real(z_array), np.imag(z_array), label=f'Lift in z space)')
axs[0].set_title( 'z space')

axs[1].plot( np.real(m_array), np.imag(m_array), label=f'Path in m space)')
axs[1].set_title( 'm space')

plt.legend()
plt.show()

In [ ]:
# Find the z_target such that M(z_target) = m_target
empirical_mean = zeroes_first_kind.mean()
radius = 1.4
phase  = -np.pi/2 
interval = np.linspace( phase, phase + 2*np.pi, 100 )
ellipse  = np.cos( interval ) + 1.0*np.sin( interval )*1j
m_array = radius*ellipse
z_array = []
debug = False
for m in m_array:
    m_target  = np.array( [m] )
    z_initial = empirical_mean/m_target
    if len(z_array)>0:
        z_initial = z_array[-1]
    else:
        z_initial = z_final
    z_result  = newton_raphson( lambda z : M_empirical(z)-m_target, M_prime_empirical, z_initial)
    z_array.append( z_result )
    if debug:
        print( "Result: ", z_result)
        print( "Error: ", np.abs( m_target - M_empirical(z_result)) )
z_array = np.array( z_array )

# Plots in z and m space
fig, axs = plt.subplots(1, 2, figsize=(20, 10), sharey=False)

axs[0].plot( np.real(z_array), np.imag(z_array), label=f'Lift in z space)')
axs[0].scatter( np.real(critical_points), np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(critical_points), -np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (N={N})')
axs[0].scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (N-1={N-1})')
axs[0].set_title( 'z space')

axs[1].plot( np.real(m_array), np.imag(m_array), label=f'Path in m space)')
axs[1].scatter( np.real(branch_points),  np.imag(branch_points), marker='*', c=branch_points_color, label=f'Branch points (N-1={N-1})')
axs[1].scatter( np.real(branch_points), -np.imag(branch_points), marker='x', c=branch_points_color, label=f'(Conjugate) Branch points (N-1={N-1})')
axs[1].set_title( 'm space')
axs[1].set_ylim( -5, 5)

plt.legend()
plt.show()

In [ ]:
box_corners_enum = [ 'bottom_right',
             'top_right',
             'top_left',
             'bottom_left'
              ]

box_segments_enum = []
for i in range( len(box_corners_enum) ):
       current_corner = box_corners_enum[i]
       next_corner    = box_corners_enum[(i+1) % 4]
       box_segments_enum.append( (current_corner, next_corner) )
print( "Box segments enumeration: ")
print( box_segments_enum )

def box_to_path( box, mesh_size):
       interval =  np.linspace( 0,1, mesh_size)
       path = []
       for segment in box_segments_enum:
              vector = box[ segment[1] ] - box[ segment[0] ]
              origin = box[ segment[0] ]
              s = origin + interval*vector
              #
              path = path + list(s)
       return path

def extend_box( box ):
       extended_box = box
       span   = extended_box['top_left']-extended_box['bottom_right']
       height = np.abs( np.imag(span) )
       width  = np.abs( np.real(span) )
       extended_box['top_right']   = extended_box['top_left']     + width
       extended_box['bottom_left'] = extended_box['bottom_right'] - width
       extended_box['height'] = height
       extended_box['width']  = width
       return extended_box

radius = 1
box = {
       'top_left'    : 0.4 + radius*0.13j,
       'bottom_right': 7.8 - radius*0.13j,
}
box = extend_box( box )
path = box_to_path( box, 10000)
plt.plot( np.real(path), np.imag(path), c='r')
plt.show()

z_array = np.array( path )
m_array = M_empirical( z_array )

# Plots in z and m space
fig, axs = plt.subplots(1, 2, figsize=(20, 10), sharey=False)

axs[0].plot( np.real(z_array), np.imag(z_array), label=f'Lift in z space)')
axs[0].scatter( np.real(critical_points), np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(critical_points), -np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (N={N})')
axs[0].scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (N-1={N-1})')
axs[0].set_title( 'z space')

axs[1].plot( np.real(m_array), np.imag(m_array), label=f'Path in m space)')
axs[1].scatter( np.real(branch_points),  np.imag(branch_points), marker='*', c=branch_points_color, label=f'Branch points (N-1={N-1})')
axs[1].scatter( np.real(branch_points), -np.imag(branch_points), marker='x', c=branch_points_color, label=f'(Conjugate) Branch points (N-1={N-1})')
axs[1].set_title( 'm space')
axs[1].set_ylim( -5, 5)

plt.legend()
plt.show()

In [ ]:
c_interval = np.linspace(0.5, 10, 100)
s_array = (1+m_array)/(m_array*z_array)

error_array = []
for c in c_interval:
    error = np.abs( s_array*(m_array+c)-1).max()
    error_array.append( error )
error_array = np.array( error_array )
plt.plot( c_interval, error_array )
plt.show()

### Formulas

$$ m = zg - 1$$
$$ s = \frac{1+m}{m z}
   \Leftrightarrow
   sz - 1 = \frac{1}{m}
$$

In [ ]:
c = 3
s_signal_array = s_array
s_noise_array  = 1/(m_array + c)
s_deconv_array = s_signal_array/s_noise_array
m_deconv_array = 1/( s_deconv_array*z_array - 1)
m_deconv_theoretical_array = 1/(z_array-1)

# Plots in z and m space
fig, axs = plt.subplots(1, 2, figsize=(20, 10), sharey=False)

axs[0].plot( np.real(z_array), np.imag(z_array), label=f'Lift in z space)')
axs[0].scatter( np.real(critical_points), np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(critical_points), -np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (N={N})')
axs[0].scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (N-1={N-1})')
axs[0].set_title( 'z space')

axs[1].plot( np.real(m_deconv_array), np.imag(m_deconv_array), label=f'Empirical path in m space')
axs[1].plot( np.real(m_deconv_theoretical_array), np.imag(m_deconv_theoretical_array), label=f'Ground truth in m space')
axs[1].scatter( np.real(branch_points),  np.imag(branch_points), marker='*', c=branch_points_color, label=f'Branch points (N-1={N-1})')
axs[1].scatter( np.real(branch_points), -np.imag(branch_points), marker='x', c=branch_points_color, label=f'(Conjugate) Branch points (N-1={N-1})')
axs[1].set_title( 'm space')
axs[1].set_ylim( -7, 7)

plt.legend()
plt.show()

# IV. Deconvolution 

In [ ]:
g_deconv_array = (m_deconv_array+1)/z_array
g_deconv_theoretical = (m_deconv_theoretical_array+1)/z_array

space_array   = np.linspace(-1, 7, 1000)
resolution = 10.0
dz_array = z_array-np.roll(z_array, shift=1)
empirical_cdf = []
for x in space_array:
    sigmoid = 1/(1+np.exp(resolution*(z_array-x)))
    value = g_deconv_array*sigmoid*dz_array
    value = value.sum()/(2*np.pi*1.0j)
    empirical_cdf.append( value )
# end for
empirical_cdf = np.array( empirical_cdf )

plt.figure( figsize=(10,5) )
plt.plot( space_array, space_array>1.0, label='Asymptotic ground truth = Theoretical cdf of deconvolved signal' )
plt.plot( space_array, np.real(empirical_cdf), label=f'''Empirical cdf of deconvolved signal and blur at resolution {1/resolution}''' )
plt.plot( space_array+0.03, 1/(1+np.exp(resolution*(1-space_array))), label='Theoretical cdf of deconvolved signal at same resolution' )
plt.legend()
plt.show()